# Домашнее задание № 4. Языковые модели

In [4]:
!pip install razdel -q

In [12]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from nltk.tokenize import sent_tokenize
import nltk
from collections import Counter
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [2]:
with open('lenta.txt', encoding='utf-8') as f:
    news = f.read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [6]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [13]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))

In [18]:
matrix_news = lil_matrix((len(bigrams_news),
                        len(unigrams_news)))

id2unigram_news = list(unigrams_news)
id2bigram_news  = list(bigrams_news)

unigram2id_news = {unigram:i for i, unigram in enumerate(id2unigram_news)}
bigram2id_news  = {bigram :i for i, bigram  in enumerate(id2bigram_news)}

for trigram in trigrams_news:
    bigram, word = trigram.rsplit(maxsplit=1)
    matrix_news[bigram2id_news[bigram], unigram2id_news[word]] = trigrams_news[trigram] / bigrams_news[bigram]

matrix_news = csc_matrix(matrix_news)

In [19]:
def apply_temperature(probas, temperature):
    log_probas = np.log(np.maximum(probas, 1e-10))
    adjusted_log_probas = log_probas / temperature
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

In [16]:
def generate_temp(matrix, id2unigram, id2bigram, unigram2id, bigram2id, n=100, start='<start> <start>', temperature=1.):
    text = start.split()
    current_idx = bigram2id[start]

    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=apply_temperature(matrix[current_idx].toarray()[0], temperature=temperature))
        text.append(id2unigram[chosen])

        if id2unigram[chosen] == '<end>':
            current_idx = bigram2id['<start> <start>']
            text.extend(['<start>', '<start>'])
        else:
            current_idx = bigram2id[' '.join(text[-2:])]
    return ' '.join(text)

In [31]:
print(generate_temp(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, n=40, temperature=0.01).replace('<start> <start>', '').replace('<end>', '\n'))

 в настоящее время в россии 
  в настоящее время в составе делегации министр внутренних дел россии игорь иванов 
  в настоящее время в республике 
  в настоящее время в составе делегации в ходе встречи с президентом сша биллом клинтоном 
 


In [23]:
print(generate_temp(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, n=40, temperature=0.2).replace('<start> <start>', '').replace('<end>', '\n'))

 по словам представителя штаба огв части и подразделения федеральных сил 
  в настоящее время в районах населенных пунктов 
  в частности в исковом заявлении говорится что вклады граждан должны обеспечиваться путем страхования 
  в то же время в разработке кремлевских


In [24]:
print(generate_temp(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, n=40, temperature=0.8).replace('<start> <start>', '').replace('<end>', '\n'))

 обвинения могут быть рим и иерусалим именно в этой сфере больше будет построено 10 судов 
  в результате наводнений в китае официально признаны пять религий буддизм даосизм христианство конфуцианство и ислам и все необходимое чтобы защитить мирное население и работу


In [28]:
print(generate_temp(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, n=40, temperature=1.5).replace('<start> <start>', '').replace('<end>', '\n'))

 освободившиеся средства планировалось передать в центральную клиническую больницу в среду проголосовали 385 депутатов воздержавшихся и голосовавших против всех 2,76 избирателей 
  россию представляет госкомпания росвооружение 
  усилено положение о федеральном характере таможенного дела и обоснованно пришел к однозначному и единодушному


In [34]:
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    return prob, len(tokens)

def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [35]:
perp_corpus = news[5000000:]
perp_norm = normalize(perp_corpus)
perp_vocab = Counter(perp_norm)
perp_probas = Counter({word:c/len(perp_norm) for word, c in perp_vocab.items()})

In [53]:
perp_text = generate_temp(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, n=500).replace('<start> <start>', '').replace('<end>', '\n').split('\n')

In [54]:
perps = []
for text in perp_text:
    perps.append(perplexity(*compute_joint_proba(text, perp_probas)))
np.mean(perps)

4754.359638930203

## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [56]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [57]:
def generate_with_beam_search(matrix, id2unigram, id2bigram, unigram2id, bigram2id, n=100, max_beams=5, start='<start> <start>'):
    initial_node = Beam(sequence=start.split(), score=np.log1p(0))
    beams = [initial_node]

    for i in range(n):
        new_beams = []
        for beam in beams:
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue

            last_id = bigram2id[' '.join(beam.sequence[-2:])]
            probas = matrix[last_id].toarray()[0]
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                if not probas[top_id]:
                    break
                new_sequence = beam.sequence + [id2unigram[top_id]]
                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence) for beam in sorted_sequences]
    return sorted_sequences

In [59]:
generate_with_beam_search(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news)

['<start> <start> как сообщает риа новости <end>',
 '<start> <start> об этом риа новости <end>',
 '<start> <start> об этом сообщает риа новости <end>',
 '<start> <start> об этом сообщает агентство риа новости <end>',
 '<start> <start> об этом сообщает итар-тасс со ссылкой на пресс-службу мчс россии сергей шойгу <end>']

In [62]:
generate_with_beam_search(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, start='<start> летом')

['<start> летом прошлого года <end>',
 '<start> летом текущего года <end>',
 '<start> летом 1998 года <end>',
 '<start> летом 1994 года в москве <end>',
 '<start> летом прошлого года в москве <end>']

In [63]:
generate_with_beam_search(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, start='завтра утром')

['завтра утром <end>',
 'завтра утром к рассмотрению протест генпрокуратуры <end>',
 'завтра утром к рассмотрению протест генпрокуратуры на большой скорости с грузовиком газ-452 <end>',
 'завтра утром к рассмотрению протест генпрокуратуры на большой скорости врезался мини-трактор обслуживавший заправку <end>',
 'завтра утром к рассмотрению протест генпрокуратуры на большой скорости неожиданно выскочил автомобиль жигули девятой модели <end>']

In [68]:
generate_with_beam_search(matrix_news, id2unigram_news, id2bigram_news, unigram2id_news, bigram2id_news, start='речь идет о')

['речь идет о нескольких десятках новейших боевых самолетов семейства су <end>',
 'речь идет о сумме в 100 миллионов долларов <end>',
 'речь идет о сумме приблизительно в 11,5 миллионов долларов <end>',
 'речь идет о фильме снятом сотрудниками мвд и прокуратуры <end>',
 'речь идет о нескольких десятках новейших боевых самолетов и самолетов зафрахтованных гуманитарными организациями <end>']